In [1]:
import gc
import os
import numpy as np 
import pandas as pd 
import utils.paths as plh
import utils.read_utils as hlpread
from sklearn.pipeline import Pipeline
from src.models.feature_eng.TE_KFold import KFoldTargetEncoder
from src.models.feature_eng.FreqEncoding import FrequencyEncoding
from src.models.feature_eng.Combine_feature import CombineFeatures
from src.models.feature_eng.KFoldFreqEncoding import KFoldFrequencyEncoding
from src.models.feature_eng.CountVectorizerEncoding import CountVectorizerEncoding
from src.models.feature_eng.TFIDFVectorizerEncoding import TFIDFVectorizerEncoding

In [2]:
PROJECT_ROOT = plh.get_project_root()

train_data = os.path.join(PROJECT_ROOT, 
                          hlpread.read_yaml_key('data_source.data_folders'),
                          hlpread.read_yaml_key('data_source.prepared.folder'),
                          hlpread.read_yaml_key('data_source.prepared.clean_train'),
                        )
train = hlpread.read_from_parquet(train_data)

print(train.shape)

print(train.columns)
#col_use = [x for x in train.columns if not x in ['ROLE_TITLE', 'MGR_ID']]
#train = train[col_use]
#targetcol = 'ACTION'

#te_col = list(train.columns)
#te_col.remove(targetcol)

(30391, 10)
Index(['ACTION', 'RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2',
       'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY',
       'ROLE_CODE'],
      dtype='object')


In [47]:
from sklearn.preprocessing import LabelEncoder

feature_engg = Pipeline( steps = [
                                    ('combine_feature', CombineFeatures()),
                                    #('label_encoder', LabelEncoder()),
                                ]
                         ,verbose =  True
                        )  

X = feature_engg.fit_transform(train) 

X.columns        

[Pipeline] ... (step 1 of 1) Processing combine_feature, total=   0.2s


Index(['ACTION', 'RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2',
       'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY',
       'ROLE_CODE', 'RESOURCE_ROLE_ROLLUP_1', 'RESOURCE_ROLE_ROLLUP_2',
       'RESOURCE_ROLE_DEPTNAME', 'RESOURCE_ROLE_FAMILY_DESC',
       'RESOURCE_ROLE_FAMILY', 'RESOURCE_ROLE_CODE',
       'ROLE_ROLLUP_1_ROLE_ROLLUP_2', 'ROLE_ROLLUP_1_ROLE_DEPTNAME',
       'ROLE_ROLLUP_1_ROLE_FAMILY_DESC', 'ROLE_ROLLUP_1_ROLE_FAMILY',
       'ROLE_ROLLUP_1_ROLE_CODE', 'ROLE_ROLLUP_2_ROLE_DEPTNAME',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC', 'ROLE_ROLLUP_2_ROLE_FAMILY',
       'ROLE_ROLLUP_2_ROLE_CODE', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC',
       'ROLE_DEPTNAME_ROLE_FAMILY', 'ROLE_DEPTNAME_ROLE_CODE',
       'ROLE_FAMILY_DESC_ROLE_FAMILY', 'ROLE_FAMILY_DESC_ROLE_CODE',
       'ROLE_FAMILY_ROLE_CODE'],
      dtype='object')

In [48]:
Xcount = X.nunique().reset_index(name = 'uniq_count')
Xcount.sort_values(by ='uniq_count', ascending= False)

,index,uniq_count
13,RESOURCE_ROLE_FAMILY_DESC,22734
15,RESOURCE_ROLE_CODE,19043
12,RESOURCE_ROLE_DEPTNAME,17354
11,RESOURCE_ROLE_ROLLUP_2,13422
14,RESOURCE_ROLE_FAMILY,13195
10,RESOURCE_ROLE_ROLLUP_1,11091
1,RESOURCE,7518
25,ROLE_DEPTNAME_ROLE_FAMILY_DESC,4553
2,MGR_ID,4085
22,ROLE_ROLLUP_2_ROLE_FAMILY_DESC,3949


In [49]:
Xcount[Xcount['uniq_count']>=5000].sort_values(by = 'uniq_count', ascending = False)

,index,uniq_count
13,RESOURCE_ROLE_FAMILY_DESC,22734
15,RESOURCE_ROLE_CODE,19043
12,RESOURCE_ROLE_DEPTNAME,17354
11,RESOURCE_ROLE_ROLLUP_2,13422
14,RESOURCE_ROLE_FAMILY,13195
10,RESOURCE_ROLE_ROLLUP_1,11091
1,RESOURCE,7518


In [42]:
#Order of the pipeline matter as this impact the output
feature_engg = Pipeline( steps = [
                                ('combine_feature', CombineFeatures()),

                                #('frequency_encoding', FrequencyEncoding(min_group_size = 2)),

                                #('tfidf_vectorizer_encoding', TFIDFVectorizerEncoding()),

                                ('count_vectorizer_encoding', CountVectorizerEncoding()),

                                ('KFoldTE', KFoldTargetEncoder()),

                                ]
                              ,verbose =  True
                        )                        


X = feature_engg.fit_transform(train) 

X.shape

[Pipeline] ... (step 1 of 3) Processing combine_feature, total=   0.2s
[Pipeline]  (step 2 of 3) Processing count_vectorizer_encoding, total= 2.2min
[Pipeline] ........... (step 3 of 3) Processing KFoldTE, total=  42.2s


(30391, 233)

In [ ]:
X.head()

In [43]:
X.columns

Index(['ACTION', 'RESOURCE', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME',
       'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE',
       'RESOURCE_ROLE_ROLLUP_1', 'RESOURCE_ROLE_ROLLUP_2',
       ...
       'ROLE_ROLLUP_2_ROLE_DEPTNAME_Kfold_TE',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC_Kfold_TE',
       'ROLE_ROLLUP_2_ROLE_FAMILY_Kfold_TE',
       'ROLE_ROLLUP_2_ROLE_CODE_Kfold_TE',
       'ROLE_DEPTNAME_ROLE_FAMILY_DESC_Kfold_TE',
       'ROLE_DEPTNAME_ROLE_FAMILY_Kfold_TE',
       'ROLE_DEPTNAME_ROLE_CODE_Kfold_TE',
       'ROLE_FAMILY_DESC_ROLE_FAMILY_Kfold_TE',
       'ROLE_FAMILY_DESC_ROLE_CODE_Kfold_TE',
       'ROLE_FAMILY_ROLE_CODE_Kfold_TE'],
      dtype='object', length=233)

In [44]:
feature_columns = X.select_dtypes(exclude= ['object']).columns 
feature_columns[1:]

Index(['ACTION', 'RESOURCE_svd_ROLE_ROLLUP_1_cv',
       'RESOURCE_svd_ROLE_ROLLUP_1_ROLE_FAMILY_DESC_cv',
       'ROLE_ROLLUP_1_svd_RESOURCE_cv', 'ROLE_ROLLUP_1_svd_ROLE_ROLLUP_2_cv',
       'ROLE_ROLLUP_1_svd_ROLE_FAMILY_DESC_cv',
       'ROLE_ROLLUP_1_svd_ROLE_FAMILY_cv', 'ROLE_ROLLUP_1_svd_ROLE_CODE_cv',
       'ROLE_ROLLUP_1_svd_RESOURCE_ROLE_ROLLUP_1_cv',
       'ROLE_ROLLUP_1_svd_RESOURCE_ROLE_ROLLUP_2_cv',
       ...
       'ROLE_ROLLUP_2_ROLE_DEPTNAME_Kfold_TE',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC_Kfold_TE',
       'ROLE_ROLLUP_2_ROLE_FAMILY_Kfold_TE',
       'ROLE_ROLLUP_2_ROLE_CODE_Kfold_TE',
       'ROLE_DEPTNAME_ROLE_FAMILY_DESC_Kfold_TE',
       'ROLE_DEPTNAME_ROLE_FAMILY_Kfold_TE',
       'ROLE_DEPTNAME_ROLE_CODE_Kfold_TE',
       'ROLE_FAMILY_DESC_ROLE_FAMILY_Kfold_TE',
       'ROLE_FAMILY_DESC_ROLE_CODE_Kfold_TE',
       'ROLE_FAMILY_ROLE_CODE_Kfold_TE'],
      dtype='object', length=205)

In [4]:
feature_columns = X.select_dtypes(include = ['object']).columns 
feature_columns

Index(['RESOURCE', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME',
       'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE',
       'RESOURCE_ROLE_ROLLUP_1', 'RESOURCE_ROLE_ROLLUP_2',
       'RESOURCE_ROLE_DEPTNAME', 'RESOURCE_ROLE_FAMILY_DESC',
       'RESOURCE_ROLE_FAMILY', 'RESOURCE_ROLE_CODE',
       'ROLE_ROLLUP_1_ROLE_ROLLUP_2', 'ROLE_ROLLUP_1_ROLE_DEPTNAME',
       'ROLE_ROLLUP_1_ROLE_FAMILY_DESC', 'ROLE_ROLLUP_1_ROLE_FAMILY',
       'ROLE_ROLLUP_1_ROLE_CODE', 'ROLE_ROLLUP_2_ROLE_DEPTNAME',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC', 'ROLE_ROLLUP_2_ROLE_FAMILY',
       'ROLE_ROLLUP_2_ROLE_CODE', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC',
       'ROLE_DEPTNAME_ROLE_FAMILY', 'ROLE_DEPTNAME_ROLE_CODE',
       'ROLE_FAMILY_DESC_ROLE_FAMILY', 'ROLE_FAMILY_DESC_ROLE_CODE',
       'ROLE_FAMILY_ROLE_CODE'],
      dtype='object')

In [10]:
def assign_rnd_integer(dataset, number_of_times = 1, seed = 2023):
    new_dataset = pd.DataFrame()
    np.random.seed(seed)
    for c in dataset.columns:
        for i in range(number_of_times):
            col_name = c+"_"+str(i)
            unique_vals = dataset[c].unique()
            labels = np.array(list(range(len(unique_vals))))
            np.random.shuffle(labels)
            mapping = pd.DataFrame({c: unique_vals, col_name: labels})
            new_dataset[col_name] = (dataset[[c]]
                                     .merge(mapping, on = c, how = 'left')[col_name]
                                    ).values
    return new_dataset

X_2 = assign_rnd_integer(X[feature_columns])

In [12]:
X_2.columns

Index(['RESOURCE_0', 'ROLE_ROLLUP_1_0', 'ROLE_ROLLUP_2_0', 'ROLE_DEPTNAME_0',
       'ROLE_TITLE_0', 'ROLE_FAMILY_DESC_0', 'ROLE_FAMILY_0', 'ROLE_CODE_0',
       'RESOURCE_ROLE_ROLLUP_1_0', 'RESOURCE_ROLE_ROLLUP_2_0',
       'RESOURCE_ROLE_DEPTNAME_0', 'RESOURCE_ROLE_FAMILY_DESC_0',
       'RESOURCE_ROLE_FAMILY_0', 'RESOURCE_ROLE_CODE_0',
       'ROLE_ROLLUP_1_ROLE_ROLLUP_2_0', 'ROLE_ROLLUP_1_ROLE_DEPTNAME_0',
       'ROLE_ROLLUP_1_ROLE_FAMILY_DESC_0', 'ROLE_ROLLUP_1_ROLE_FAMILY_0',
       'ROLE_ROLLUP_1_ROLE_CODE_0', 'ROLE_ROLLUP_2_ROLE_DEPTNAME_0',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC_0', 'ROLE_ROLLUP_2_ROLE_FAMILY_0',
       'ROLE_ROLLUP_2_ROLE_CODE_0', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_0',
       'ROLE_DEPTNAME_ROLE_FAMILY_0', 'ROLE_DEPTNAME_ROLE_CODE_0',
       'ROLE_FAMILY_DESC_ROLE_FAMILY_0', 'ROLE_FAMILY_DESC_ROLE_CODE_0',
       'ROLE_FAMILY_ROLE_CODE_0'],
      dtype='object')

In [36]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

chi2_feature = SelectKBest(chi2, k = 29)
best_feature = chi2_feature.fit_transform(X_2, X.ACTION)


In [37]:
chi2_feature.scores_

array([  179.79681866,   440.72187967,  1500.57693636,   184.74044727,
          73.27398768,  5084.72221475,    58.63225907,   404.12521478,
        1598.98391097,    32.32397894,  1538.48661015,   754.73077548,
         588.22319832,    95.7254337 ,   450.55712416,   797.35428785,
        2218.18303481,   591.18845719,  2027.66683632,  1455.33417203,
        4834.32111978,   185.74463499,  7909.07825373,    54.62173851,
        6012.25826507,   544.87378976, 12109.88162184,  2726.38963297,
         325.85226668])

In [40]:
chi2_feature.scores_.argsort()

array([ 9, 23,  6,  4, 13,  0,  3, 21, 28,  7,  1, 14, 25, 12, 17, 11, 15,
       19,  2, 10,  8, 18, 16, 27, 20,  5, 24, 22, 26], dtype=int64)

In [39]:
chi2_feature.feature_names_in_

array(['RESOURCE_0', 'ROLE_ROLLUP_1_0', 'ROLE_ROLLUP_2_0',
       'ROLE_DEPTNAME_0', 'ROLE_TITLE_0', 'ROLE_FAMILY_DESC_0',
       'ROLE_FAMILY_0', 'ROLE_CODE_0', 'RESOURCE_ROLE_ROLLUP_1_0',
       'RESOURCE_ROLE_ROLLUP_2_0', 'RESOURCE_ROLE_DEPTNAME_0',
       'RESOURCE_ROLE_FAMILY_DESC_0', 'RESOURCE_ROLE_FAMILY_0',
       'RESOURCE_ROLE_CODE_0', 'ROLE_ROLLUP_1_ROLE_ROLLUP_2_0',
       'ROLE_ROLLUP_1_ROLE_DEPTNAME_0',
       'ROLE_ROLLUP_1_ROLE_FAMILY_DESC_0', 'ROLE_ROLLUP_1_ROLE_FAMILY_0',
       'ROLE_ROLLUP_1_ROLE_CODE_0', 'ROLE_ROLLUP_2_ROLE_DEPTNAME_0',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC_0', 'ROLE_ROLLUP_2_ROLE_FAMILY_0',
       'ROLE_ROLLUP_2_ROLE_CODE_0', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_0',
       'ROLE_DEPTNAME_ROLE_FAMILY_0', 'ROLE_DEPTNAME_ROLE_CODE_0',
       'ROLE_FAMILY_DESC_ROLE_FAMILY_0', 'ROLE_FAMILY_DESC_ROLE_CODE_0',
       'ROLE_FAMILY_ROLE_CODE_0'], dtype=object)

In [41]:
chi2_feature.feature_names_in_[chi2_feature.scores_.argsort()]

array(['RESOURCE_ROLE_ROLLUP_2_0', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_0',
       'ROLE_FAMILY_0', 'ROLE_TITLE_0', 'RESOURCE_ROLE_CODE_0',
       'RESOURCE_0', 'ROLE_DEPTNAME_0', 'ROLE_ROLLUP_2_ROLE_FAMILY_0',
       'ROLE_FAMILY_ROLE_CODE_0', 'ROLE_CODE_0', 'ROLE_ROLLUP_1_0',
       'ROLE_ROLLUP_1_ROLE_ROLLUP_2_0', 'ROLE_DEPTNAME_ROLE_CODE_0',
       'RESOURCE_ROLE_FAMILY_0', 'ROLE_ROLLUP_1_ROLE_FAMILY_0',
       'RESOURCE_ROLE_FAMILY_DESC_0', 'ROLE_ROLLUP_1_ROLE_DEPTNAME_0',
       'ROLE_ROLLUP_2_ROLE_DEPTNAME_0', 'ROLE_ROLLUP_2_0',
       'RESOURCE_ROLE_DEPTNAME_0', 'RESOURCE_ROLE_ROLLUP_1_0',
       'ROLE_ROLLUP_1_ROLE_CODE_0', 'ROLE_ROLLUP_1_ROLE_FAMILY_DESC_0',
       'ROLE_FAMILY_DESC_ROLE_CODE_0', 'ROLE_ROLLUP_2_ROLE_FAMILY_DESC_0',
       'ROLE_FAMILY_DESC_0', 'ROLE_DEPTNAME_ROLE_FAMILY_0',
       'ROLE_ROLLUP_2_ROLE_CODE_0', 'ROLE_FAMILY_DESC_ROLE_FAMILY_0'],
      dtype=object)

In [19]:
best_feature

array([[   24,   711, 10830, ...,  1433,  1956,  1345],
       [   87,    84,  2256, ...,   991,  2015,  1598],
       [  152,  1274,  8552, ...,  1366,  1316,  1315],
       ...,
       [   58,  1223,  2026, ...,  1051,  2504,  1180],
       [   96,  2021,  4739, ...,  1338,  1845,  2511],
       [  138,  1806,  5550, ...,   861,  1540,   506]])

In [ ]:
#Export the result of the dvc exp to cvs file 
"""
import csv
import subprocess
 
## call date command ##
p = subprocess.Popen("dvc exp show -A --csv", stdout = subprocess.PIPE, shell = True)
(output, err) = p.communicate()
p_status = p.wait()

with open('Example.csv', 'w', encoding='UTF8', newline='') as file:

    writer = csv.writer(file)

    header = []
    data = []
    for i, line in enumerate(output.splitlines()):

        line = line.decode('ASCII') #Output return is  “byte string“. Note: ‘b‘ character before a string is used to specify the string as a “byte string“
        
        if len(line) == 0:
            continue
        
        if len(header) == 0:
            for word in line.split(','):
                header.append(word)

        else:
            row = []    
            for word in line.split(','):
                row.append(word)

            data.append(row)
    
    writer.writerow(header)
    writer.writerow(data)


#f = pd.DataFrame(data)
#f.drop(range(146,198,1), axis = 1, inplace = True)
#f.columns = header
#f.shape
#

"""

# Evaluate Models #

Evaluate the models on the saved test dataset 

## Decision Tree ##

In [ ]:
trained_model = hlpread.read_object(
                                    os.path.join(PROJECT_ROOT, 
                                                 hlpread.read_yaml_key('data_source.data_folders'),
                                                 hlpread.read_yaml_key('model.trained_model')
                                                 )
                                    )

In [ ]:
Y_predictions_by_class = trained_model.predict_proba(X).astype(float) #Return 2d numpy array which is the probaility for each class label
Y = Y.astype(float)

In [ ]:
Y_pred = Y_predictions_by_class.argmax(-1)  
conf_matrix = confusion_matrix(Y, Y_pred)

print(conf_matrix)

In [ ]:
f1_score(Y, Y_pred)

## Random Forest ##

In [ ]:
trained_model = hlpread.read_object(
                                    os.path.join(PROJECT_ROOT, 
                                                 hlpread.read_yaml_key('data_source.data_folders'),
                                                 hlpread.read_yaml_key('model.trained_model')
                                                 )
)
                                    

In [ ]:
Y_predictions_by_class = trained_model.predict_proba(X).astype(float) #Return 2d numpy array which is the probaility for each class label
Y = Y.astype(float)

In [ ]:
Y_pred = Y_predictions_by_class.argmax(-1)  
conf_matrix = confusion_matrix(Y, Y_pred)

print(conf_matrix)

In [ ]:
f1_score(Y, Y_pred)

## Extra Decision Tree(Ensemble) ## 

In [ ]:
trained_model = hlpread.read_object(
                                    os.path.join(PROJECT_ROOT, 
                                                 hlpread.read_yaml_key('data_source.data_folders'),
                                                 hlpread.read_yaml_key('model.trained_model')
                                                 )
)
                                    

In [ ]:
Y_predictions_by_class = trained_model.predict_proba(X).astype(float) #Return 2d numpy array which is the probaility for each class label
Y = Y.astype(float)

Y_pred = Y_predictions_by_class.argmax(-1)  
conf_matrix = confusion_matrix(Y, Y_pred)

print(conf_matrix)


In [ ]:
f1_score(Y, Y_pred)

## Logistic Regression ##

In [ ]:
trained_model = hlpread.read_object(
                                    os.path.join(PROJECT_ROOT, 
                                                 hlpread.read_yaml_key('data_source.data_folders'),
                                                 hlpread.read_yaml_key('model.trained_model')
                                                 )
)
                                    

In [ ]:
Y_predictions_by_class = trained_model.predict_proba(X).astype(float) #Return 2d numpy array which is the probaility for each class label
Y = Y.astype(float)

Y_pred = Y_predictions_by_class.argmax(-1)  
conf_matrix = confusion_matrix(Y, Y_pred)

print(conf_matrix)


In [ ]:
f1_score(Y, Y_pred)

# Cross Validation for Decision Tree model #

Let's perform cross validation on the final parmater of the decision tree model

In [ ]:
training_param =  hlpread.read_yaml_key('trained_model')

In [ ]:
#Load clean data
import utils.read_utils as hlpread

clean_train_data = os.path.join(PROJECT_ROOT,
                                hlpread.read_yaml_key('data_source.data_folders'),
                                hlpread.read_yaml_key('data_source.prepared.folder'),
                                hlpread.read_yaml_key('data_source.prepared.clean_train'),
                                )
db_train = hlpread.read_from_parquet(clean_train_data)

In [ ]:
# Create pipeline
from src.models.feature_eng.CountVectorizerEncoding import CountVectorizerEncoding
from src.models.feature_eng.Combine_feature import CombineFeatures
from src.models.feature_eng.TE_KFold import KFoldTargetEncoder
from sklearn.pipeline import Pipeline

feature_engg = Pipeline(steps = [
                                    ('combine_feature', CombineFeatures()),
                                    ('count_vectorizer_encoding', CountVectorizerEncoding()),
                                    ('KFoldTE', KFoldTargetEncoder())
                                ]) 

X = feature_engg.fit_transform(db_train) 

In [ ]:
#Define cross validation DT
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit

cv_results = {}
split_params = hlpread.read_yaml_key('train_test_split')
split_s = StratifiedShuffleSplit(n_splits = 4, test_size = split_params['test_size'], random_state = split_params['random_seed'])

fold = 0
for train_index, test_index in split_s.split(X[X.columns[30:]], X['ACTION']):
    
    Y_train, Y_test = X[['ACTION']].iloc[train_index,:], X[['ACTION']].iloc[test_index,:]
    X_train, X_test = X[X.columns[30:]].iloc[train_index,:], X[X.columns[30:]].iloc[test_index,:]

    model = DecisionTreeClassifier(criterion = 'gini', random_state = 42)
    model.set_params(**training_param['params'])

    model.fit(X_train, Y_train ) 

    Y_test_pred = model.predict_proba(X_test).astype(float)
    auc_score = roc_auc_score(Y_test.astype(float), Y_test_pred[:,1])

    cv_results[fold] = auc_score
    fold += 1

In [ ]:
cv_results

In [ ]:
sum(cv_results.values()) / len(cv_results)

## Train model ##

In [ ]:
model.fit(X[X.columns[30:]], X['ACTION'])

In [ ]:
Y_predictions_by_class = model.predict_proba(X[X.columns[30:]]).astype(float) #Return 2d numpy array which is the probaility for each class label
Y = X['ACTION'].astype(float)

Y_pred = Y_predictions_by_class.argmax(-1)  
conf_matrix = confusion_matrix(Y, Y_pred)

print(conf_matrix)

In [ ]:
auc_score

In [ ]:

from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(Y, Y_predictions_by_class[:,1])
roc_auc_curve_df = pd.DataFrame()    
roc_auc_curve_df['false_positive_rates'] = fpr
roc_auc_curve_df['true_positive_rates'] = tpr
roc_auc_curve_df['thresholds'] = thresholds

fig, ax = plt.subplots(figsize = (7.5, 7.5))
ax.plot(roc_auc_curve_df['false_positive_rates'], roc_auc_curve_df['true_positive_rates'],  color = 'green', label = 'ROC Curve') #, marker = 'o'

ax.tick_params(axis = 'both', labelcolor = 'green')
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
label_str = str.format('ROC-AUC: {0}',  round(auc_score, 3))
ax.text(0.5, 0, label_str, fontsize = 6)